In [1]:
!wget https://tmpfiles.org/dl/1499103/final.xlsx-sheet1.csv

--2023-06-05 03:16:56--  https://tmpfiles.org/dl/1499103/final.xlsx-sheet1.csv
Resolving tmpfiles.org (tmpfiles.org)... 172.67.195.247, 104.21.21.16, 2606:4700:3030::6815:1510, ...
Connecting to tmpfiles.org (tmpfiles.org)|172.67.195.247|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 634583 (620K) [text/plain]
Saving to: ‘final.xlsx-sheet1.csv’

final.xlsx-sheet1.c 100%[===================>] 619.71K  --.-KB/s    in 0.1s    

2023-06-05 03:16:56 (6.22 MB/s) - ‘final.xlsx-sheet1.csv’ saved [634583/634583]



In [2]:
!pip install shortuuid
!pip install schedule

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import pandas as pd
import numpy as np
import json
import random
import shortuuid

def generate_dataset(file_name):
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(file_name)
    df['itemID'] = [shortuuid.uuid() for _ in range(len(df))]

    # Now you can work with the DataFrame 'df'
    df = df[['itemID', 'description', 'category', 'city']]

    # Your JSON data
    json_data = [
        {
            "userId": "user1",
            "clickedItems": random.choices(df['itemID'], k=50)
        },
        {
            "userId": "user2",
            "clickedItems": random.choices(df['itemID'], k=50)
        },
        {
            "userId": "user3",
            "clickedItems": random.choices(df['itemID'], k=50)
        },
        {
            "userId": "user4",
            "clickedItems": random.choices(df['itemID'], k=50)
        },
        {
            "userId": "user5",
            "clickedItems": random.choices(df['itemID'], k=50)
        }
    ]

    # Convert JSON to DataFrame
    data = []
    for user in json_data:
        for item in user['clickedItems']:
            data.append([user['userId'], item, 1])
    df_user_clicked = pd.DataFrame(data, columns=['userID', 'itemID', 'clicked'])

    # Create a DataFrame of all possible user-item pairs
    all_users = df_user_clicked['userID'].unique()
    all_items = df['itemID']

    data_all = []
    for user in all_users:
        for item in all_items:
            data_all.append([user, item])

    df_all = pd.DataFrame(data_all, columns=['userID', 'itemID'])

    # Merge df_all with df_items to add category, description, and other_attribute
    df_all = pd.merge(df_all, df, on='itemID', how='left')

    # Merge the user clicked data onto the DataFrame of all user-item pairs
    df_final = pd.merge(df_all, df_user_clicked, how='left', on=['userID', 'itemID'], suffixes=('', '_user_clicked'))

    # If the user has clicked the item, replace the 'clicked' value in df_all with the one from df_user_clicked
    df_final['clicked'].fillna(0, inplace=True)  # fill NaNs with 0

    return df_final

df = generate_dataset('/content/final.xlsx-sheet1.csv')

df

,userID,itemID,description,category,city,clicked
0,user1,3TTYRU8bnMw2dx8gRWyXjP,Air terjun Gitgit adalah air terjun yang terle...,Cagar Alam,Buleleng,0.0
1,user1,FRvNNt3ekX6N7Z82ispjnG,Air terjun Tegenungan adalah air terjun yang t...,Cagar Alam,Denpasar,0.0
2,user1,LJRcJdz6nXs2or2Vh32K6n,Alun-Alun Purworejo adalah sebuah alun-alun at...,Taman Hiburan,Purworejo,0.0
3,user1,9WHhAhPoqoAjToH7HwHxAd,Bali Safari & Marine Park (BSMP) merupakan tem...,Taman Hiburan,Gianyar,0.0
4,user1,9cA56ytodmECruyHvAwS7Z,Batu Secret Zoo merupakan tempat wisata dan ke...,Cagar Alam,Batu,0.0
...,...,...,...,...,...,...
4495,user5,HVU3Z2RAeDKABnWZzHqLNc,"Sejak diresmikan pada bulan Desember 2017, Atl...",Taman Hiburan,Surabaya,0.0
4496,user5,2QyjoWCxSEMK4yTfwzM7sH,Taman Hiburan Rakyat atau THR tentunya sudah t...,Taman Hiburan,Surabaya,0.0
4497,user5,9RGDnb7QXWWViH5d47p5tx,Air mancur menari atau dancing fountain juga a...,Taman Hiburan,Surabaya,0.0
4498,user5,mYmtqbN6oRYEvnzNubmRY7,Taman Flora adalah salah satu taman kota di Su...,Taman Hiburan,Surabaya,0.0


In [18]:
import pandas as pd
import random
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Split the data into a training set and a validation set
df_train, df_val = train_test_split(df, test_size=0.2, random_state=42)

# Define the maximum number of words in the texts to keep based on word frequency
max_words = 500

# Tokenizers
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(df_train['description'])

tokenizer_categories = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer_categories.fit_on_texts(df_train['category'])

tokenizer_other = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer_other.fit_on_texts(df_train['city'])

# Convert the texts to sequences
description_sequences_train = tokenizer.texts_to_sequences(df_train['description'])
description_sequences_val = tokenizer.texts_to_sequences(df_val['description'])

categories_sequences_train = tokenizer_categories.texts_to_sequences(df_train['category'])
categories_sequences_val = tokenizer_categories.texts_to_sequences(df_val['category'])

other_sequences_train = tokenizer_other.texts_to_sequences(df_train['city'])
other_sequences_val = tokenizer_other.texts_to_sequences(df_val['city'])

# Pad the sequences so they are all the same length
description_padded_train = pad_sequences(description_sequences_train, maxlen=max_words)
description_padded_val = pad_sequences(description_sequences_val, maxlen=max_words)

categories_padded_train = pad_sequences(categories_sequences_train, maxlen=max_words)
categories_padded_val = pad_sequences(categories_sequences_val, maxlen=max_words)

other_padded_train = pad_sequences(other_sequences_train, maxlen=max_words)
other_padded_val = pad_sequences(other_sequences_val, maxlen=max_words)

# Custom Label Encoding for user_id and item_id
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

encoded_user_ids_train = user_encoder.fit_transform(df_train['userID'])
encoded_item_ids_train = item_encoder.fit_transform(df_train['itemID'])

encoded_user_ids_val = user_encoder.transform(df_val['userID'])
encoded_item_ids_val = item_encoder.transform(df_val['itemID'])

labels_train = df_train['clicked']
labels_val = df_val['clicked']

# Build the model
user_input = layers.Input(shape=(1,), name='user')
item_input = layers.Input(shape=(1,), name='item')
description_input = layers.Input(shape=(max_words,), name='description')
category_input = layers.Input(shape=(max_words,), name='category')
other_input = layers.Input(shape=(max_words,), name='other')

user_embedding = layers.Embedding(input_dim=len(user_encoder.classes_), output_dim=50)(user_input)
item_embedding = layers.Embedding(input_dim=len(item_encoder.classes_), output_dim=50)(item_input)
description_embedding = layers.Embedding(input_dim=max_words, output_dim=50)(description_input)
category_embedding = layers.Embedding(input_dim=max_words, output_dim=50)(category_input)
other_embedding = layers.Embedding(input_dim=max_words, output_dim=50)(other_input)

user_embedding = layers.Flatten()(user_embedding)
item_embedding = layers.Flatten()(item_embedding)
description_embedding = layers.GlobalAveragePooling1D()(description_embedding)
category_embedding = layers.GlobalAveragePooling1D()(category_embedding)
other_embedding = layers.GlobalAveragePooling1D()(other_embedding)

concatenated = layers.Concatenate()([user_embedding, item_embedding, description_embedding, category_embedding, other_embedding])

dense1 = layers.Dense(128, activation='relu')(concatenated)
dense2 = layers.Dense(64, activation='relu')(dense1)
out = layers.Dense(1, activation='sigmoid')(dense2)

model = tf.keras.Model(inputs=[user_input, item_input, description_input, category_input, other_input], outputs=out)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([encoded_user_ids_train, encoded_item_ids_train, description_padded_train, categories_padded_train, other_padded_train], labels_train, epochs=10, validation_data=([encoded_user_ids_val, encoded_item_ids_val, description_padded_val, categories_padded_val, other_padded_val], labels_val))

# Save the model, label encoders, and tokenizers for future use
model.save('recommendation_model.h5')
np.save('user_encoder_classes.npy', user_encoder.classes_)
np.save('item_encoder_classes.npy', item_encoder.classes_)
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('tokenizer_categories.pickle', 'wb') as handle:
    pickle.dump(tokenizer_categories, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('tokenizer_other.pickle', 'wb') as handle:
    pickle.dump(tokenizer_other, handle, protocol=pickle.HIGHEST_PROTOCOL)


Epoch 1/10
113/113 [==============================] - 3s 11ms/step - loss: 0.2627 - accuracy: 0.9442 - val_loss: 0.2112 - val_accuracy: 0.9456
Epoch 2/10
113/113 [==============================] - 1s 9ms/step - loss: 0.1919 - accuracy: 0.9442 - val_loss: 0.2283 - val_accuracy: 0.9456
Epoch 3/10
113/113 [==============================] - 1s 9ms/step - loss: 0.1407 - accuracy: 0.9442 - val_loss: 0.2811 - val_accuracy: 0.9456
Epoch 4/10
113/113 [==============================] - 1s 9ms/step - loss: 0.1240 - accuracy: 0.9442 - val_loss: 0.3560 - val_accuracy: 0.9456
Epoch 5/10
113/113 [==============================] - 1s 13ms/step - loss: 0.1212 - accuracy: 0.9442 - val_loss: 0.3817 - val_accuracy: 0.9456
Epoch 6/10
113/113 [==============================] - 1s 10ms/step - loss: 0.1222 - accuracy: 0.9442 - val_loss: 0.3774 - val_accuracy: 0.9456
Epoch 7/10
113/113 [==============================] - 1s 9ms/step - loss: 0.1195 - accuracy: 0.9444 - val_loss: 0.3868 - val_accuracy: 0.9444
Epo

In [44]:
import schedule
import time
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

class Recommender:
    def __init__(self, model_path, user_encoder_path, item_encoder_path, tokenizer_path, tokenizer_categories_path, tokenizer_other_path, df, max_words):
        self.model_path = model_path
        self.user_encoder_path = user_encoder_path
        self.item_encoder_path = item_encoder_path
        self.tokenizer_path = tokenizer_path
        self.tokenizer_categories_path = tokenizer_categories_path
        self.tokenizer_other_path = tokenizer_other_path
        self.df = df
        self.max_words = max_words
        self.reload_model()

    def reload_model(self):
        self.model = tf.keras.models.load_model(self.model_path)

        self.user_encoder = LabelEncoder()
        self.user_encoder.classes_ = np.load(self.user_encoder_path, allow_pickle=True)

        self.item_encoder = LabelEncoder()
        self.item_encoder.classes_ = np.load(self.item_encoder_path, allow_pickle=True)

        with open(self.tokenizer_path, 'rb') as handle:
            self.tokenizer = pickle.load(handle)

        with open(self.tokenizer_categories_path, 'rb') as handle:
            self.tokenizer_categories = pickle.load(handle)

        with open(self.tokenizer_other_path, 'rb') as handle:
            self.tokenizer_other = pickle.load(handle)

    def predict(self, new_user_id):
        all_item_ids = self.df['itemID'].unique().tolist()
        all_categories = []
        all_descriptions = []
        all_other_attributes = []

        for item_id in all_item_ids:
            item_data = self.df[self.df['itemID'] == item_id].iloc[0]
            all_categories.append(item_data['category'])
            all_descriptions.append(item_data['description'])
            all_other_attributes.append(item_data['city'])

        if new_user_id not in self.user_encoder.classes_:
            print("New user detected. Assigning random existing user for prediction.")
            new_user_id = np.random.choice(self.user_encoder.classes_)

        encoded_new_user_id = self.user_encoder.transform([new_user_id]*len(all_item_ids))
        encoded_all_item_ids = self.item_encoder.transform(all_item_ids)

        description_sequences = self.tokenizer.texts_to_sequences(all_descriptions)
        description_padded = pad_sequences(description_sequences, maxlen=self.max_words)

        category_sequences = self.tokenizer_categories.texts_to_sequences(all_categories)
        category_padded = pad_sequences(category_sequences, maxlen=self.max_words)

        other_sequences = self.tokenizer_other.texts_to_sequences(all_other_attributes)
        other_padded = pad_sequences(other_sequences, maxlen=self.max_words)

        predictions = self.model.predict([encoded_new_user_id, encoded_all_item_ids, description_padded, category_padded, other_padded])
        top_10_indices = np.argsort(predictions[:, 0])[-10:]

        print("Top 10 recommendations for", new_user_id, "are:")
        for index in reversed(top_10_indices):
            print(f'Item: {all_item_ids[index]}, predicted click probability: {predictions[index][0]}')

    # Dummy implementation
    def recommend_popular_items(self):
        print("Here are some popular items")



rec = Recommender('recommendation_model.h5', 'user_encoder_classes.npy', 'item_encoder_classes.npy', 'tokenizer.pickle', 'tokenizer_categories.pickle', 'tokenizer_other.pickle', df, 500)

def job():
    rec.reload_model()

# Schedule the task every day at 12am
schedule.every().day.at("00:00").do(job)

#while True:
#    schedule.run_pending()
#    time.sleep(1)

rec.predict("budiman")

New user detected. Assigning random existing user for prediction.
29/29 [==============================] - 0s 3ms/step
Top 10 recommendations for user3 are:
Item: P8Hs4z8WJaYxNYMeQ56QVX, predicted click probability: 0.6135902404785156
Item: HVU3Z2RAeDKABnWZzHqLNc, predicted click probability: 0.6119400262832642
Item: Dz2gmzdhxgGjmcg8dPdt6R, predicted click probability: 0.596519410610199
Item: S4ELmMr3KkfJtzQG7wNeLu, predicted click probability: 0.5855777859687805
Item: RgmQr8fpKQ7yFfcYE6iPpG, predicted click probability: 0.5715683698654175
Item: 4HSauRSDAnkntYQhiPU9Da, predicted click probability: 0.5561433434486389
Item: Kr5JWkeCbcKsda42T5Ybwz, predicted click probability: 0.5489989519119263
Item: WxRitkumUWJdPkL9vykHSm, predicted click probability: 0.5320135354995728
Item: 4nqpDRmYff64cFeGuaYiWn, predicted click probability: 0.5265135765075684
Item: apKjahSWG3Yd2j5kSvVvtv, predicted click probability: 0.5114524960517883
